Import the libraries

In [43]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.datasets import mnist

Loading dataset

In [44]:
(x_train , y_train) , (x_test , y_test) = mnist.load_data()

Selecting a subset of data

In [45]:
x_train = x_train[:200]

In [46]:
y = y_train[:200]

In [47]:
X = x_train.T
X = X/255

In [48]:
y.resize((200,1))
y = y.T

Checking values

In [49]:
pd.Series(y[0]).value_counts()

1    26
9    23
7    21
4    21
3    21
0    21
2    20
6    19
8    15
5    13
dtype: int64

converting into binary classification 

In [50]:
for i  in range(y.shape[1]):
  if y[0][i] > 4:
    y[0][i] = 1
  else:
    y[0][i] = 0


Chceking values counts

In [51]:
pd.Series(y[0]).value_counts()

0    109
1     91
dtype: int64

Initializing filter

In [52]:
f = np.random.uniform(size = (3,5,5))
f = f.T
print('Filter 1','\n',f[:,:,0], '\n' )
print('Filter 2','\n',f[:,:,1], '\n' )
print('Filter 3','\n',f[:,:,2], '\n' )


Filter 1 
 [[0.86588584 0.71485102 0.87521954 0.09804079 0.06050066]
 [0.14546274 0.507431   0.50735416 0.87637052 0.52588684]
 [0.95320896 0.02676922 0.78578819 0.94176799 0.42114781]
 [0.96964406 0.65903576 0.27781796 0.79368924 0.54711545]
 [0.53159531 0.74716899 0.97034762 0.02192238 0.33639692]] 

Filter 2 
 [[0.60805739 0.98096657 0.92773345 0.65931079 0.03819589]
 [0.99610089 0.18503006 0.53623186 0.78944866 0.31018941]
 [0.63776169 0.85540424 0.20838207 0.77557098 0.66669818]
 [0.44138798 0.272597   0.93457065 0.86490068 0.60227867]
 [0.87426864 0.4783617  0.38655166 0.95630636 0.40253281]] 

Filter 3 
 [[0.05419892 0.11420265 0.70366215 0.79298456 0.00219316]
 [0.66363017 0.61085552 0.24177697 0.61263503 0.06736363]
 [0.27277627 0.30611602 0.82537662 0.5444801  0.3332066 ]
 [0.96704575 0.99715898 0.30702831 0.49361862 0.27935872]
 [0.57739187 0.46319738 0.05670943 0.05523783 0.22371955]] 



In [53]:
X.shape , y.shape ,f.shape

((28, 28, 200), (1, 200), (5, 5, 3))

Patches of the same dimension

In [54]:
# Generating patches from images
new_image = []

#for number of images
for k in range(X.shape[2]):
  #sliding in horizontal direction
  for i in range(X.shape[0]-f.shape[0]+1):
    #sliding in vertical direction
    for j in range(X.shape[1]-f.shape[1]+1):
      new_image.append(X[:,:,k][i:i+f.shape[0],j:j+f.shape[1]])

#reizing the generated patches as per number of images

new_image = np.array(new_image)
new_image.resize((X.shape[2],int(new_image.shape[0]/X.shape[2]),new_image.shape[1],new_image.shape[2]))
new_image.shape

(200, 576, 5, 5)

Let us initialize the weight matrix

In [55]:
#number of features in data set

s_row = X.shape[0] - f.shape[0]+1
s_col = X.shape[1] - f.shape[1]+1
num_filter = f.shape[2]

inputlayer_neurons = (s_row)*(s_col)*(num_filter)
output_neurons = 1

#initializing weight
wo = np.random.uniform(size = (inputlayer_neurons,output_neurons))


Convolution neural network

In [56]:
#defining the sigmoid function
def sigmoid (x):
  return 1/(1+np.exp(-x))

#derivative of sigmoid function
def derivative_sigmoid(x):
  return x*(1-x)

Apply the Sigmoid activation function

In [57]:
#generating output of convolution layer
filter_output = []

#for each image

for i in range(len(new_image)):
  #apply each filter 
  for k in range(f.shape[2]):
    #do element wise multiplication
    for j in range(new_image.shape[1]):
      filter_output.append((new_image[i][j]*f[:,: ,k]).sum())

filter_output = np.resize(np.array(filter_output),(len(new_image),f.shape[2],new_image.shape[1]))

#applying actiavtion over convolution output
filter_output_sigmoid = sigmoid(filter_output)

filter_output.shape,filter_output_sigmoid.shape

((200, 3, 576), (200, 3, 576))

Applying the linear transformation and activation function on this data

In [58]:
#generating input for fully connected layer
filter_output_sigmoid = filter_output_sigmoid.reshape((
filter_output_sigmoid.shape[0],
filter_output_sigmoid.shape[1]  * filter_output_sigmoid.shape[2]
))
filter_output_sigmoid = filter_output_sigmoid.T

# linaer transformation for fully connected layer
output_layer_input = np.dot(wo.T,filter_output_sigmoid)
output_layer_input =(output_layer_input-np.average(output_layer_input))/np.std(output_layer_input)

#activation function
output = sigmoid(output_layer_input)

Backward Propagation

In [59]:
#error 
error = np.square(y-output)/2

#error w.r.t output (Gradient)
error_wrt_output = -(y-output)

#error w.r.t sigmoaid transformation (output_layer_input)
output_wrt_output_layer_input = output * (1-output)

#error w.r.t weight
output_wrt_w = filter_output_sigmoid


Derivative of backward propagation

In [60]:
#Error w.r.t sigmoid output
output_layer_input_wrt_filter_output_sigmoid = wo.T

#Erroe w.r.t sigmoid transformation
filter_output_sigmoid_wrt_filter_output = filter_output_sigmoid * (1-filter_output_sigmoid)


#calculating derivatives for backprop convolution

error_wrt_filter_output = np.dot(output_layer_input_wrt_filter_output_sigmoid.T,
                                 error_wrt_output * output_wrt_output_layer_input)*filter_output_sigmoid_wrt_filter_output

error_wrt_filter_output = np.average(error_wrt_filter_output ,axis = 1)

error_wrt_filter_output = np.resize(error_wrt_filter_output,
                                    (X.shape[0]-f.shape[0]+1,X.shape[1]-f.shape[1]+1,f.shape[2]))



Convolution Process

In [61]:
filter_update = []
for i in range(f.shape[2]):
  for j in range(f.shape[0]):
    for k in range(f.shape[1]):
      temp = 0
      spos_row = j
      spos_col = k
      epos_row = spos_row + s_row
      epos_col = spos_col + s_col
      for l in range(X.shape[2]):
        temp = temp +  X[spos_row:epos_row,spos_col : epos_col,1]*error_wrt_filter_output[:,:,i].sum() 
filter_update.append(temp/X.shape[2])
filter_update_array = np.array(filter_update)
filter_update_array = np.resize(filter_update_array ,(f.shape[2],f.shape[0],f.shape[1]))

Update the error

In [66]:
for i in range(f.shape[2]):
  f[:,:,i] = f[:,:,i] -  lr * filter_update_array[i]

NameError: ignored